In [130]:
from transformers import Swinv2Config, Swinv2Model, AutoFeatureExtractor
import transformers
import os, json, glob, requests, torch, cv2
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image
from transformers import Swinv2ForImageClassification

In [95]:
from transformers import AutoFeatureExtractor

ImportError: cannot import name 'Swinv2FeatureExtractor' from 'transformers' (c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\transformers\__init__.py)

In [90]:
#set default path
TRAIN_JSON_PATH = 'data/label/train/'
VALID_JSON_PATH = 'data/label/val/'

TRAIN_IMG_PATH = 'data/train/'
VALID_IMG_PATH = 'data/val/'

TRAIN_SEGMENT_PATH = 'data/segmentation/train/'
VALID_SEGMENT_PATH = 'data/segmentation/val/'

IMG_ROWS = 256
IMG_COLS = 256

In [60]:
#To get json list that contains info about the pictures.
def get_json_list_1d(json_folder_path): #returns a 1 dim array
    temp_loader = []
    json_train = []
    emotion_list_json = os.listdir(json_folder_path)

    for emotion in emotion_list_json:
        with open (json_folder_path + emotion, "r", encoding='euc-kr') as f:
            temp_loader.append(json.load(f))     

    for i in range(len(temp_loader)):
        json_train += temp_loader[i]     

    return json_train


def get_json_list_2d(json_folder_path): #returns 2 dim array
    json_train = []
    emotion_list_json = os.listdir(json_folder_path)

    for emotion in emotion_list_json:
        with open (json_folder_path + emotion, "r", encoding='euc-kr') as f:
            json_train.append(json.load(f))     

    return json_train





In [72]:
#call for the functions
json_train = get_json_list_1d(TRAIN_JSON_PATH)
json_val = get_json_list_1d(VALID_JSON_PATH)
print(json_train[0].keys()) #see what keys are available
print(len(json_train))
print(len(json_val))

#test
# json_train[0]['annot_A']['boxes']

# for coord in json_train[0]['annot_A']['boxes']:
#     print(json_train[0]['annot_A']['boxes'][coord])

dict_keys(['filename', 'gender', 'age', 'isProf', 'faceExp_uploader', 'bg_uploader', 'annot_A', 'annot_B', 'annot_C'])
5382
1049


In [62]:
json_train_2d = get_json_list_2d(TRAIN_JSON_PATH)
json_val_2d = get_json_list_2d(VALID_JSON_PATH)
#0 = 분노, 1 = 기쁨, 2 = 당황, 3 = 슬픔
#0 = anger, 1 = happy, 2 = panic, 3 = sadness

print(os.listdir(TRAIN_IMG_PATH))

['anger', 'happy', 'panic', 'sadness']


In [63]:
print(len(json_val_2d[1]))
# len(json_train)
print(len(os.listdir(VALID_IMG_PATH + 'happy/')))

303
303


파일 JSON 이랑 비교해서 빠진거 있나 확인

In [13]:
filenames_json = [entry['filename'] for entry in json_val_2d[1]]
filenames_folder = os.listdir(VALID_IMG_PATH + 'happy/')


print(list( set(filenames_json) - set(filenames_folder)))
print(len(list( set(filenames_json) - set(filenames_folder))))

[]
0


In [14]:
#checking if that file exist in the path
temp = r'C:\Users\User\Downloads\Project3\val\happy'
filenames_folder = os.listdir(temp)

'7uji1c8ab7778bc286c3cd90077847f9a906f00b210838ba57877a29abd60ffix.jpg' in filenames_folder

False

get all images and put them into a list

In [116]:
#0 = 분노, 1 = 기쁨, 2 = 당황, 3 = 슬픔

# feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")

#designed for 2D
def load_image_and_label_2D(json_list, img_path):
    emotion_list = os.listdir(img_path)
    img_file = []
    label = []
    
    eng_to_num = {'anger'   : '0',
                 'happy'   : '1',
                 'panic'   : '2',
                 'sadness' : '3'             
                 }
    i = 0
    for emotion in emotion_list:
        for entry in range(len(json_list[i])):
            try:
                img_dir = img_path + emotion + '/' + json_list[i][entry]['filename']
                img = np.array(image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS)))/255
                img_file.append(img)
                label.append(int(eng_to_num[emotion]))
            except Exception:
                print("Emotion " + emotion + " " + json_list[i][entry]['filename'] + " Not available")
                pass
        i += 1
    return img_file, label



In [143]:
#0 = 분노, 1 = 기쁨, 2 = 당황, 3 = 슬픔

# feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")

#designed for normal json file
def load_image_and_label_1D(json_list, img_path):
    img_file = []
    label_list = []
    
    eng_to_num = {'분노'   : '0',
                 '기쁨'   : '1',
                 '당황'   : '2',
                 '슬픔' : '3'             
                 }
    kr_to_en = {'분노'   : 'anger',
                 '기쁨'   : 'happy',
                 '당황'   : 'panic',
                 '슬픔' : 'sadness'             
                 }

    
    for entry in range(len(json_list)):
        try:
            label = json_list[entry]['faceExp_uploader']
            img_dir = img_path + kr_to_en[label] + '/' + json_list[entry]['filename']
            # img = np.array(image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS)))/255
            img = image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS))
            img_file.append(img)
            label_list.append(int(eng_to_num[label]))
        except Exception:
            print("Emotion " + label + " " + json_list[entry]['filename'] + " Not available")
            pass

    return img_file, label_list

In [144]:
train_img, train_label = load_image_and_label_1D(json_train, TRAIN_IMG_PATH)

Emotion 당황 twdva8d723865fa1005873a6967797c75f3dd83197f13f7daa3e3926947071al6.jpg Not available
Emotion 당황 5yid6d7ae32bf14b1343da46cfdb3fdffc8e1c64baf4fb1b175690f648a870v69.jpg Not available
Emotion 당황 bpv531e5cebdb1c00085158bb0677c5596ccabf1c8a5cb0007991571bf306wcd4.jpg Not available
Emotion 당황 s1q397a38900c1e1f950496109900e7c01f0630459790d8590010655c4090vlh0.jpg Not available
Emotion 당황 7m2k46c05803855399ea1ee1911875ee59e05640c93135c0bc91b5660456cdo8k.jpg Not available
Emotion 당황 n55yc741f09701506c7eb5ba30533db35c905ab5bf0597b5652ece03c5950ozv1.jpg Not available
Emotion 당황 bid7b3c93409e1d83514689337f6d3d63a338339d71c9ce7fded153d3576fk2ml.jpg Not available
Emotion 당황 if0lfc4b517c6597e3656dbea350e406e1b5ae4e175e460597064875909d9t40p.jpg Not available
Emotion 당황 58w153f3f5ade2ad20ec19e6744ff2ff0a54ba3520ffbef36f3f3a47af75fxuf5.jpg Not available
Emotion 당황 lhyp01dff4faf13d1b4319722dc31bb3a104a3878ab3816a03bb8acdfa3d3eomk.jpg Not available
Emotion 당황 vba0d31d23b44711180462388c8aba4424a0aa2

In [117]:
# train_img, train_label = load_image_and_label_2D(json_train_2d, TRAIN_IMG_PATH)


Emotion panic twdva8d723865fa1005873a6967797c75f3dd83197f13f7daa3e3926947071al6.jpg Not available
Emotion panic 5yid6d7ae32bf14b1343da46cfdb3fdffc8e1c64baf4fb1b175690f648a870v69.jpg Not available
Emotion panic bpv531e5cebdb1c00085158bb0677c5596ccabf1c8a5cb0007991571bf306wcd4.jpg Not available
Emotion panic s1q397a38900c1e1f950496109900e7c01f0630459790d8590010655c4090vlh0.jpg Not available
Emotion panic 7m2k46c05803855399ea1ee1911875ee59e05640c93135c0bc91b5660456cdo8k.jpg Not available
Emotion panic n55yc741f09701506c7eb5ba30533db35c905ab5bf0597b5652ece03c5950ozv1.jpg Not available
Emotion panic bid7b3c93409e1d83514689337f6d3d63a338339d71c9ce7fded153d3576fk2ml.jpg Not available
Emotion panic if0lfc4b517c6597e3656dbea350e406e1b5ae4e175e460597064875909d9t40p.jpg Not available
Emotion panic 58w153f3f5ade2ad20ec19e6744ff2ff0a54ba3520ffbef36f3f3a47af75fxuf5.jpg Not available
Emotion panic lhyp01dff4faf13d1b4319722dc31bb3a104a3878ab3816a03bb8acdfa3d3eomk.jpg Not available
Emotion panic vba0d3

In [118]:
val_img, val_label = load_image_and_label_2D(json_val_2d, VALID_IMG_PATH)


In [112]:
print(len(train_img))
print(type(train_label[0]))
print(len(val_img))
print(train_img[0].shape)

5364
<class 'str'>
1049
(256, 256, 3)


Modeling

In [135]:
model_name = "microsoft/swinv2-tiny-patch4-window8-256"
image_processor = AutoFeatureExtractor.from_pretrained(model_name)
# model = Swinv2ForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True, num_labels=4)
model = Swinv2ForImageClassification.from_pretrained(model_name)

c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [136]:
inputs = image_processor(train_img, return_tensors="pt", do_rescale=False)

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


In [145]:
with torch.no_grad():
    logits = model(**inputs).logits

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 8436842496 bytes.

In [ ]:
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

In [101]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [125]:
type(train_dataset)

list

In [126]:
train_dataset = (train_img, train_label)
val_dataset = (val_img, val_label)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [127]:
def train_model(model, train_loader, val_loader, epochs=10, lr=1e-5):
    optimizer = optim.RMSprop(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        val_preds, val_targets = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).logits
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = outputs.argmax(dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(val_targets, val_preds)

        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Train Loss: {train_loss:.4f}')
        print(f'Val Loss: {val_loss:.4f}')
        print(f'Val Accuracy: {val_accuracy:.4f}')

    return model

trained_model = train_model(model, train_loader, val_loader, epochs=10, lr=1e-5)

TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
# df = pd.DataFrame(json_train, columns=['filename', 'faceExp_uploader'])
# df['faceExp_uploader'].unique()
# train_npz = np.load(TRAIN_SEGMENT_PATH + 'train_anger.npz')
# file_name = json_train[0]['filename']
# plt.imshow(train_npz[file_name], vmin=0, vmax=5)